<a href="https://colab.research.google.com/github/Maheshkumarcseb/MY_PROJECTS/blob/main/SimilarImageGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import gradio as gr
import requests
import torch
import clip
from PIL import Image
from io import BytesIO

# ✅ Load CLIP Model (Vision-Language Model)
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# ✅ Google Custom Search API Details (Replace with Your Own API Key & CSE ID)
GOOGLE_API_KEY = "your_api_key"
GOOGLE_CSE_ID = "your_cse_id"

# ✅ Function to Extract CLIP Image Features
def get_image_embedding(image):
    """Extract CLIP embedding from an image."""
    image = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(image)
    return embedding

# ✅ Get Best Matching Text for an Image
def get_best_text_description(image_embedding):
    """Find the most relevant text description using CLIP."""
    possible_labels = [
        "shoes", "sneakers", "sandals", "boots", "heels",
        "bags", "handbags", "backpacks", "wallets",
        "t-shirts", "jeans", "jackets", "sweaters", "shorts",
        "hats", "caps", "gloves", "scarves"
    ]

    text_inputs = torch.cat([clip.tokenize(label) for label in possible_labels]).to(device)

    with torch.no_grad():
        text_embeddings = model.encode_text(text_inputs)

    similarities = (image_embedding @ text_embeddings.T).squeeze(0)
    best_match_idx = similarities.argmax().item()
    return possible_labels[best_match_idx]

# ✅ Fetch Images from Google Custom Search
def fetch_google_images(query):
    """Fetch similar images from Google Custom Search API."""
    url = f"https://www.googleapis.com/customsearch/v1?q={query}&searchType=image&key={GOOGLE_API_KEY}&cx={GOOGLE_CSE_ID}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return [item["link"] for item in data.get("items", [])]
    return []

# ✅ Find Similar Images
def find_similar_images(uploaded_image):
    """Process image, get description, and fetch similar images."""
    image = Image.open(uploaded_image).convert("RGB")
    image_embedding = get_image_embedding(image)

    # Generate the best matching text label
    query_text = get_best_text_description(image_embedding)

    # Fetch images based on the detected category
    similar_images = fetch_google_images(query_text)

    return similar_images if similar_images else ["No similar images found."]

# ✅ Gradio UI
iface = gr.Interface(
    fn=find_similar_images,
    inputs=gr.Image(type="filepath"),
    outputs=gr.Gallery(label="Similar Images"),
    title="Visual Search with VLM (CLIP)",
    description="Upload an image, and the system will find visually similar images from the web.",
)

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f5e5b7648dc45cda7e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
